In [1]:
import tkinter as tk
import serial
import time
import threading
import winsound

In [2]:
arduino = serial.Serial("COM3", 9600)
time.sleep(2)

# Función para ejecutar un beep del sistema, como alarma cuando llegue a rojo
def beep():
    winsound.Beep(1000, 300)  # Frecuencia: 1000 Hz, Duración: 300 ms

# Función para encender el flujo de agua
def encender():
    arduino.write(b'a')

# Función para apagar el flujo de agua    
def apagar():
    arduino.write(b'q')
    
# Función para cambiar el color del círculo
def cambiar_color(color):
    canvas.itemconfig(circulo, fill=color)
    
    
# Función para leer datos del sensor en un hilo separado
def leer_sensor():
    while True:
        try:
            data = arduino.readline().decode().strip()
            if data.isdigit():
                distancia = int(data)
                
                if distancia < 2:
                    beep()  # Distancia límite, apagado de emergencia
                    apagar()
                elif distancia < 10:
                    cambiar_color("red")  # Distancia corta, colorear de rojo
                elif distancia < 20:
                    cambiar_color("yellow")  # Distancia media, colorear de amarillo
                else:
                    cambiar_color("green")  # Distancia segura, colorear de verde 
        except:
            pass
        time.sleep(0.5)
   
# Creación de la ventana principal
window = tk.Tk()
window.title("Control de la bomba de agua")
window.geometry("600x300")
window.configure(bg="lightgray") # Color de fondo

# Crear un lienzo para dibujar un círculo
canvas = tk.Canvas(window, width=100, height=100, bg="black", highlightthickness=0)
canvas.place(x=175, y=20)

# Dibujar el círculo inicial (rojo por defecto)
circulo = canvas.create_oval(10, 10, 90, 90, fill="red")

bttn = tk.Button(window, text="Encender", command=encender, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttn.place(x=100, y=200)

bttn = tk.Button(window, text="Apagar", command=apagar, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttn.place(x=250, y=200)

# Iniciar el hilo para leer datos del sensor y no bloquear la interfaz gráfica
threading.Thread(target=leer_sensor, daemon=True).start()

window.mainloop()